# Learn pandas (31/12/2025)

In [22]:
import pandas as pd
import numpy as np

### Dataset của bạn

Cấu trúc

- Duration (int) – thời gian tập
- Pulse (int) – nhịp tim trung bình
- Maxpulse (int) – nhịp tim tối đa
- Calories (float, có missing values)

In [4]:
url = "./Data/data.csv"

### Bài 1

#### Đọc file data.csv vào DataFrame

In ra:
- 5 dòng đầu
- 5 dòng cuối
- số dòng, số cột
- kiểu dữ liệu từng cột

In [5]:
metadata = pd.read_csv(url)
metadata.head()
metadata.tail()
metadata.index
metadata.columns
metadata.dtypes

Duration      int64
Pulse         int64
Maxpulse      int64
Calories    float64
dtype: object

### Bài 2

1. Lấy cột Calories theo 2 cách khác nhau
2. Lấy dòng thứ 10 bằng iloc
3. Lấy giá trị Calories của dòng thứ 10

In [6]:
metadata.Calories
metadata["Calories"]
metadata.iloc[9]
metadata.loc[9, 'Calories']


np.float64(269.0)

### Bài 3

1. Kiểm tra xem mỗi cột có bao nhiêu giá trị bị thiếu
2. Cho biết tổng số dòng có Calories bị thiếu

In [7]:
for label in metadata.columns.tolist():
    print(f"So dong bi thieu cua cot {label}: {len(metadata[metadata[label].isnull()])} dong")

So dong bi thieu cua cot Duration: 0 dong
So dong bi thieu cua cot Pulse: 0 dong
So dong bi thieu cua cot Maxpulse: 0 dong
So dong bi thieu cua cot Calories: 5 dong


### Bài 4 – Lọc dữ liệu

- Lọc các buổi tập có: ```Duration >= 60```
- Lọc các buổi tập có: ```Calories > 400```
- Lọc các buổi tập có: ```Pulse > 100 và Calories > 300```

In [8]:
len(metadata[metadata["Duration"] >= 60])
len(metadata[metadata["Calories"] >= 60])
metadata[(metadata["Pulse"] > 100) & (metadata["Calories"] > 300)]

,Duration,Pulse,Maxpulse,Calories
0,60,110,130,409.1
1,60,117,145,479.0
2,60,103,135,340.0
4,45,117,148,406.0
6,60,110,136,374.0
...,...,...,...,...
155,60,111,151,368.5
158,60,114,150,382.8
166,60,115,145,310.2
167,75,120,150,320.4


### Bài 5 – Xử lý missing data

1. Tạo bản sao DataFrame
2. Điền các giá trị Calories bị thiếu bằng:
    - giá trị trung bình

3. Kiểm tra lại còn missing values hay không

In [20]:
data = metadata.copy()
data.loc[data["Calories"].isna(), "Calories"] = data["Calories"].mean()
print(data)

     Duration  Pulse  Maxpulse  Calories
0          60    110       130     409.1
1          60    117       145     479.0
2          60    103       135     340.0
3          45    109       175     282.4
4          45    117       148     406.0
..        ...    ...       ...       ...
164        60    105       140     290.8
165        60    110       145     300.0
166        60    115       145     310.2
167        75    120       150     320.4
168        75    125       150     330.4

[169 rows x 4 columns]


### Bài 6 – Thêm cột

- Tạo cột mới ```Intensity```:
    - ```"Low"``` nếu ```Pulse < 100```
    - ```"Medium"``` nếu ```100 ≤ Pulse ≤ 120```
    - ```"High"``` nếu ```Pulse > 120```

In [ ]:
data["Intensity"] = np.where(
    metadata["Pulse"] < 100, "Low", 
    np.where(metadata["Pulse"] <= 120, "Media", "High")
)
print(data)

     Duration  Pulse  Maxpulse  Calories Intensity
0          60    110       130     409.1     Media
1          60    117       145     479.0     Media
2          60    103       135     340.0     Media
3          45    109       175     282.4     Media
4          45    117       148     406.0     Media
..        ...    ...       ...       ...       ...
164        60    105       140     290.8     Media
165        60    110       145     300.0     Media
166        60    115       145     310.2     Media
167        75    120       150     320.4     Media
168        75    125       150     330.4      High

[169 rows x 5 columns]


### Bài 7

Tính:

- Calories trung bình
- Calories lớn nhất
- Calories nhỏ nhấ 

In [28]:
data["Calories"].mean()
data["Calories"].max()
data["Calories"].min()

np.float64(50.3)

### Bài 8 – GroupBy

1. Group theo ```Duration```

2. Với mỗi Duration, tính:
    - Calories trung bình
    - Pulse trung bình
3. Sắp xếp kết quả theo Calories giảm dần

In [39]:
result = data.groupby("Duration", as_index=False).agg(
    mean_pulse=("Pulse", "mean"),
    mean_calories = ("Calories", "mean")
).sort_values(by="Duration", ascending=False)

print(result)

    Duration  mean_pulse  mean_calories
15       300  108.000000    1500.200000
14       270  100.000000    1729.000000
13       210  122.500000    1618.200000
12       180   93.666667     733.600000
11       160  109.500000     943.700000
10       150  101.500000     939.400000
9        120  100.000000     666.833333
8         90   93.750000     541.800000
7         80  123.000000     643.100000
6         75  122.500000     325.400000
5         60  106.126582     341.046465
4         45  107.485714     279.096585
3         30  109.812500     192.125000
2         25  152.000000     244.200000
1         20  125.000000     151.600000
0         15  102.000000      87.350000


### Bài 9 – Phân tích cường độ tập

1. Group theo Intensity

2. Tính:
    - Số buổi tập
    - Calories trung bình
    - Maxpulse trung bình

In [45]:
result_2 = data.groupby("Intensity").agg(
    training=("Duration", "count"),
    mean_calories=("Calories", "mean"),
    mean_maxpulse = ("Maxpulse", "mean")
)
print(result_2)

           training  mean_calories  mean_maxpulse
Intensity                                        
High             21     371.104762     157.142857
Low              40     337.849512     119.125000
Media           108     390.753433     135.083333
